# Week 7 — Part 01: CLI design (argparse) + good defaults

**Estimated time:** 60–90 minutes

## What success looks like (end of Part 01)

- `--help` clearly documents required inputs, defaults, and outputs.
- Incorrect usage fails fast with an actionable error message.
- You can normalize and validate key args (e.g., `output_dir`, `model`, `seed`).

### Checkpoint

After running this notebook, you should be able to:

- build a parser with stable flag names (`--input`, `--output_dir`, `--model`, `--seed`)
- show `normalize_args(...)` and `validate_args(...)` running without crashing

## Learning Objectives

- Treat a CLI as an explicit contract for users
- Implement a clean `argparse` interface with good defaults
- Validate required inputs and fail with helpful errors
- Keep flags consistent across scripts

## Overview

Your CLI is an interface like an API.

A good CLI:

- makes correct usage easy
- makes incorrect usage obvious
- documents itself via `--help`

---

## Underlying theory: a CLI is a contract over user behavior

Users (including “future you”) will misuse your tool. A CLI contract reduces ambiguity by making:

- valid usage explicit (required flags, clear help)
- invalid usage fast-failing (clear errors)
- outputs predictable (stable locations, stable schemas)

Good defaults are not just convenience; they are a *policy* that encodes the “safe/typical” way to run your system.

In [ ]:
import argparse
from pathlib import Path


def build_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(
        prog="run_capstone",
        description="Run the capstone pipeline with stable outputs",
    )
    p.add_argument("--input", required=True, help="Input CSV file path")
    p.add_argument("--output_dir", default="output", help="Artifact directory")
    p.add_argument("--model", default="llama3.1", help="Model name")
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--config", default=None, help="Optional config file")
    return p


def require_file(path: str) -> Path:
    p = Path(path).expanduser()
    if not p.exists():
        raise FileNotFoundError(f"Input file not found: {p}. Try --help for usage.")
    if p.stat().st_size == 0:
        raise ValueError(f"Input file is empty: {p}. Provide a non-empty CSV.")
    return p


parser = build_parser()
print("CLI parser ready")

In [ ]:
from typing import Tuple


def normalize_args(output_dir: str, model: str) -> Tuple[str, str]:
    # TODO: implement normalization (e.g., strip whitespace, enforce lowercase model).
    out = (output_dir or "").strip()
    mdl = (model or "").strip()
    return out, mdl


def validate_args(output_dir: str, seed: int) -> None:
    # TODO: add validation (e.g., seed must be >= 0, output_dir not empty).
    if not output_dir:
        raise ValueError("output_dir must be non-empty")


out_dir_norm, model_norm = normalize_args(" output ", " LLaMA3.1 ")
validate_args(out_dir_norm, 42)
print("normalized:", out_dir_norm, model_norm)

print("Implement normalize_args() and validate_args().")

## Self-check

- Can a teammate run `python run_capstone.py --help` and understand how to use it?

## References

- Python `argparse`: https://docs.python.org/3/library/argparse.html
- Click: https://click.palletsprojects.com/

## Appendix: Solutions (peek only after trying)

Reference implementations for `normalize_args` and `validate_args`.

In [ ]:
def normalize_args(output_dir: str, model: str) -> Tuple[str, str]:
    out = (output_dir or "").strip()
    mdl = (model or "").strip().lower()
    return out, mdl


def validate_args(output_dir: str, seed: int) -> None:
    if not output_dir or not str(output_dir).strip():
        raise ValueError("output_dir must be non-empty")
    if int(seed) < 0:
        raise ValueError("seed must be >= 0")


print("solution_example:", normalize_args(" output ", " LLaMA3.1 "))
validate_args("output", 0)